In [1]:
import pandas as pd

# READ DATASETS

In [3]:
folder = "./datasets/"


### STATIONS DATASET


In [6]:
df_stations = pd.read_csv(folder + "DB_estaciones.txt", sep=";", header=0)

In [7]:

df = pd.read_csv(folder + "datos_temp.csv", header=0).drop(['Unnamed: 0'], axis=1)

In [10]:
df.head()

,Unnamed: 0,id,codi_estacio,codi_variable,data_lectura,valor_lectura
0,0,CA320101090000,CA,32,2009-01-01T00:00:00.000,6.0
1,1,CC320101090000,CC,32,2009-01-01T00:00:00.000,4.6
2,2,CD320101090000,CD,32,2009-01-01T00:00:00.000,3.4
3,3,CE320101090000,CE,32,2009-01-01T00:00:00.000,7.3
4,4,CG320101090000,CG,32,2009-01-01T00:00:00.000,5.0
